In [ ]:
from geostat import GP, Mesh, NormalizingFeaturizer
import geostat.covfunc as cf
import matplotlib.pyplot as pp
import numpy as np

# Overview

In this notebook we will:
  * Use a Gaussian process with a complex stacked covariance function to generate synthetic data.
  * Use Gaussian processes with covariance functions of increasing complexity to infer the geospatial parameters from the synthetic data. Each time, log likelihood improves and the nugget decreases. A smaller nugget means that predictions are more confident.

# Synthesizing data

We will synthesize data at random locations in 3D space near the origin.

In [ ]:
locs = np.random.normal(size=[500, 3]) * [1., 1., 0.333]

There will be a depth trend, but no horizontal trends:

In [ ]:
def trend_terms(x, y, z): return z, z*z, z*z*z

Create a featurizer that the Gaussian process class `GP` will use to convert locations into trend features:

In [ ]:
featurizer = NormalizingFeaturizer(trend_terms, locs.reshape([-1, 3]))

The covariance function will be a combination of two gamma-exponentials, one which respects depth with z-anisotropy, and one that ignores depth altogether. We will set the `range` for both to be the same parameter to show that it is possible to tie parameters together.

In [ ]:
covariance = \
    cf.GammaExponential(range='r', sill='s1', gamma='g1', scale=[1., 1., 'zs']) + \
    cf.GammaExponential(range='r', sill='s2', gamma='g2', scale=[1., 1., 0.]) + \
    cf.Noise()

Instantiate a `GP` and immediately call `generate` to generate synthetic observations.
  * `parameter` holds the geostatistical parameters named above.
  * `alpha` parameterizes the normal distribution prior for trend coefficients. 

In [ ]:
obs = GP(featurizer = featurizer,
         covariance = covariance,
         parameters = dict(zs=10., r=0.33, s1=1., s2=0.5, g1=1., g2=0.5, nugget=0.25),
         hyperparameters = dict(alpha=0.1),
         verbose=True).generate(locs)

When the data is plotted, you can see an overall trend with some localized variations.

In [ ]:
fig, axs = pp.subplots(3, figsize=(7, 6), dpi=120, sharex=True, sharey=True)
vmin, vmax = obs.min(), obs.max()
pane = np.round((locs[:, 1] + 2) / 2).astype(int)
for i, ymid in enumerate(np.linspace(-2, 2, 3)):
    ymin, ymax = ymid - 1, ymid + 1
    c = axs[i].scatter(locs[pane == i, 0], locs[pane == i, 2], c=obs[pane == i], vmin=vmin, vmax=vmax)
    axs[i].set_title('y = %0.1f' % ymid)
    axs[i].set_aspect(0.9)
axs[2].set_xlabel('x-axis')
axs[1].set_ylabel('z-axis')

fig.subplots_adjust(right=0.9)
cbar_ax = fig.add_axes([0.88, 0.1, 0.02, 0.8])
fig.colorbar(c, cax=cbar_ax)

fig.suptitle('Synthetic data, projected to nearest cross section')
pp.show()

Before we continue, let's define a function that takes a model and plots predictions for the three slices shown above.

In [ ]:
def plot(gp):
    fig, axs = pp.subplots(3, figsize=(7, 6), dpi=120, sharex=True, sharey=True)
    for i, ymid in enumerate(np.linspace(-2, 2, 3)):

        mesh = Mesh.from_bounds([-3, -1, 3, 1], nx=200)
        mesh_locs = mesh.locations(proj=[[1, 0, 0], [0, 0, 1], [0, ymid, 0]]) # [x, z, 1] -> [x, y, z].
        mean, var = gp.predict(locs, obs, mesh_locs)
        meshx, meshy, out = mesh.slice(mean)
        c = axs[i].pcolormesh(meshx, meshy, out, vmin=vmin, vmax=vmax)

        axs[i].set_title('y = %0.1f' % ymid)
        axs[i].set_aspect(0.9)
    axs[2].set_xlabel('x-axis')
    axs[1].set_ylabel('z-axis')

    fig.subplots_adjust(right=0.9)
    cbar_ax = fig.add_axes([0.88, 0.1, 0.02, 0.8])
    fig.colorbar(c, cax=cbar_ax)

    fig.suptitle('Predictions for 3 cross sections')
    pp.show()

# Model 1: Bayesian regression

First let's try modeling the data with a Bayesian regression, which is a model with just trends and uncorrelated noise.

In [ ]:
covariance = cf.Noise()

gp1 = GP(featurizer = featurizer,
        covariance = covariance,
        parameters = dict(range=1.0, sill=0.5, nugget=0.5),
        hyperparameters = dict(alpha=obs.ptp()**2, reg=0, train_iters=500),
        verbose=True).fit(locs, obs)

And here are predictions for the three slices shown above.

In [ ]:
plot(gp1)

# Model 2: GP with isotropic sq-exp covariance function

Now let's layer on an isotropic squared exponential covariance function to the above model.

In [ ]:
covariance = \
    cf.SquaredExponential() + \
    cf.Noise()

gp2 = GP(featurizer = featurizer,
        covariance = covariance,
        parameters = dict(range=1.0, sill=0.5, nugget=0.5),
        hyperparameters = dict(alpha=obs.ptp()**2, reg=0, train_iters=500),
        verbose=True).fit(locs, obs)

The log-likelihood is improved as a result of a more complex model. Nugget is much lower. Predictions:

In [ ]:
plot(gp2)

# Model 3: GP with anisotropic sq-exp covariance function

Now we switch from isotropic to anisotropic for the covariance function.

In [ ]:
covariance = \
    cf.SquaredExponential(scale=[1., 1., 'zs']) + \
    cf.Noise()

gp3 = GP(featurizer = featurizer,
        covariance = covariance,
        parameters = dict(range=1.0, sill=0.5, nugget=0.5, zs=5.0),
        hyperparameters = dict(alpha=obs.ptp()**2, reg=0, train_iters=500),
        verbose=True).fit(locs, obs)

Log-likelihood and nugget both improve further. Predictions:

In [ ]:
plot(gp3)

# Model 4: GP with anisotropic gamma-exp covariance function

Now we switch from a squared-exponential to a gamma-exponential covariance function, which has an extra shape parameter.

In [ ]:
covariance = \
    cf.GammaExponential(scale=[1., 1., 'zs']) + \
    cf.Noise()

gp4 = GP(featurizer = featurizer,
        covariance = covariance,
        parameters = dict(range=1.0, sill=0.5, nugget=0.5, zs=5.0, gamma=1.0),
        hyperparameters = dict(alpha=obs.ptp()**2, reg=0, train_iters=500),
        verbose=True).fit(locs, obs)

The log-likelihood has improved very slightly but nugget improves more significantly, since the pointy peak in the gamma-exponential does some of the work of a nugget. Predictions:

In [ ]:
plot(gp4)

# Model 5: GP with stacked covariance functions

Finally we switch to using the same covariance function used to generate the synthetic data.

In [ ]:
covariance = \
    cf.GammaExponential(range='r', sill='s1', gamma='g1', scale=[1., 1., 'zs']) + \
    cf.GammaExponential(range='r', sill='s2', gamma='g2', scale=[1., 1., 0.]) + \
    cf.Noise() 

gp5 = GP(featurizer = featurizer,
        covariance = covariance,
        parameters = dict(zs=5., r=0.5, s1=2., s2=1., g1=1., g2=1., nugget=1.),
        hyperparameters = dict(alpha=obs.ptp()**2, reg=0, train_iters=500),
        verbose=True).fit(locs, obs)

Not surprisingly, log likelihood and nugget both improve further. You can see faint vertical stripes that correspond the the "depth-invariant" component of the covariance function.

In [ ]:
plot(gp5)